# SB1 Big Data Downloads: I-405 Skirball
An analysis into the corridor analysis data downloads from Streetlight and Replica

In [1]:
import pandas as pd
from siuba import *
import ast


from calitp_data_analysis.sql import to_snakecase

import altair as alt
from calitp_data_analysis import calitp_color_palette as cp

from IPython.display import Markdown, HTML, display_html, display
from IPython.core.display import display

import numpy as np

/tmp/ipykernel_469/2202862553.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
import _utils

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)

In [4]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/corridor_study_data/"

### Read in Replica Data Spring 2023

In [5]:
replica_trips = "replica-405_skirball-05_24_24-trips_dataset.csv"

In [6]:
r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))

/tmp/ipykernel_469/2643890524.py:1: DtypeWarning: Columns (18,19,20,25,26,28,29,30,31,33,36,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))


In [7]:
display(HTML("<h4><strong> Sample of data</strong></h4>"))


In [8]:
r_trips.sample(5)

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
314707,8505387565750628825,"1 (Tract 1349.07, Los Angeles, CA)","1349.07 (Los Angeles, CA)","Los Angeles County, CA",California,"4 (Tract 6037.02, Los Angeles, CA)","6037.02 (Los Angeles, CA)","Los Angeles County, CA",California,private_auto,social,home,13:31:00,14:24:59,53,31.2,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,mixed_use,multi_family,single_family,single_family,13590816709645713585,15626229210687624776,63.0,male,white_not_hispanic_or_latino,employed,in_person,7224.0,private_auto,2.0,23805.0,two,core,naics713940,multiple_units,not_attending_school,some_college,renter,indo_european,"1 (Tract 1349.07, Los Angeles, CA)","1349.07 (Los Angeles, CA)","Los Angeles County, CA",California,"3 (Tract 7004, Los Angeles, CA)","7004 (Los Angeles, CA)","Los Angeles County, CA",California
101372,12431969978777812348,"1 (Tract 1174.08, Los Angeles, CA)","1174.08 (Los Angeles, CA)","Los Angeles County, CA",California,"3 (Tract 2735.02, Los Angeles, CA)","2735.02 (Los Angeles, CA)","Los Angeles County, CA",California,private_auto,shop,home,17:16:00,18:01:55,45,20.4,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,single_family,single_family,retail,retail,3550536633128717092,6516243422276820403,62.0,female,asian_not_hispanic_or_latino,employed,in_person,27836.0,private_auto,4.0,110187.0,two,core,naics621111,single_family,graduate,bachelors_degree,owner,other,"1 (Tract 1174.08, Los Angeles, CA)","1174.08 (Los Angeles, CA)","Los Angeles County, CA",California,"3 (Tract 5435.03, Los Angeles, CA)","5435.03 (Los Angeles, CA)","Los Angeles County, CA",California
237178,1520438980495662157,"4 (Tract 1175.30, Los Angeles, CA)","1175.30 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 9800.28, Los Angeles, CA)","9800.28 (Los Angeles, CA)","Los Angeles County, CA",California,private_auto,work,home,07:19:00,08:13:36,54,23.1,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,multi_family,multi_family,industrial,industrial,8591229506040203519,16428891936194534442,24.0,male,hispanic_or_latino_origin,employed,in_person,40595.0,private_auto,4.0,102068.0,three_plus,core,naics44_45,multiple_units,not_attending_school,some_college,renter,spanish,"4 (Tract 1175.30, Los Angeles, CA)","1175.30 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 9800.28, Los Angeles, CA)","9800.28 (Los Angeles, CA)","Los Angeles County, CA",California
31761,11036673894851484580,"1 (Tract 6506.06, Los Angeles, CA)","6506.06 (Los Angeles, CA)","Los Angeles County, CA",California,"2 (Tract 56, Kern, CA)","56 (Kern, CA)","Kern County, CA",California,private_auto,work,social,07:00:00,09:53:47,173,142.9,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,multi_family,multi_family,retail,retail,673878670446905249,1964084087473365831,42.0,female,asian_not_hispanic_or_latino,employed,in_person,23942.0,private_auto,2.0,80947.0,one,core,naics23,multiple_units,not_attending_school,bachelors_degree,re

In [9]:
display(HTML("<h4><strong> Columns in Replica Trips Data</strong></h4>"))


In [10]:
r_trips.columns

Index(['activity_id', 'origin_bgrp_2020', 'origin_trct_2020',
       'origin_cty_2020', 'origin_st_2020', 'destination_bgrp_2020',
       'destination_trct_2020', 'destination_cty_2020', 'destination_st_2020',
       'primary_mode', 'trip_purpose', 'previous_trip_purpose',
       'trip_start_time', 'trip_end_time', 'trip_duration_minutes',
       'trip_distance_miles', 'vehicle_type', 'vehicle_fuel_type',
       'transit_submode', 'transit_agency', 'transit_route', 'origin_land_use',
       'origin_building_use', 'destination_land_use',
       'destination_building_use', 'trip_taker_person_id',
       'trip_taker_household_id', 'trip_taker_age', 'trip_taker_sex',
       'trip_taker_race_ethnicity', 'trip_taker_employment_status',
       'trip_taker_wfh', 'trip_taker_individual_income',
       'trip_taker_commute_mode', 'trip_taker_household_size',
       'trip_taker_household_income', 'trip_taker_available_vehicles',
       'trip_taker_resident_type', 'trip_taker_industry',
       'tri

#### Data Exploration: Replica Trips
* Summarizing the data that we exported from Replica
* Existing visualizations in replica: 
   * Primary Mode
   * Trip Purpose
   * Starting hour 
   * Trip Duration/Distance
   * Origin Destination
   * Vehicle Fuel type
   * Transit Routes/Stops/Sub mode/Agency
   * Household Income
   * Race and Ethnicity
   * Private Auto Availability
   * Age
   * Employment/School Status
   
   

In [11]:
trips = r_trips>>group_by(_.primary_mode)>>summarize(avg_trip_time = _.trip_duration_minutes.mean(),
                                                     number_trips = _.activity_id.nunique())

In [12]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("avg_trip_time"),
            color=alt.Color("avg_trip_time", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)
            ), tooltip=trips.columns.tolist())
      .properties(title = "Average Trip Time",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [13]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("number_trips"),
            color=alt.Color("number_trips", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,)
            ), tooltip=trips.columns.tolist())
      .properties(title="Number of Trips",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [14]:
display(HTML("<h4><strong>Length of Trips dataframe</strong></h4>"))

In [15]:
len(r_trips)

466756

In [16]:
display(HTML("<h4><strong>Number of unique activity ids in data</strong></h4>"))

In [17]:
r_trips>>summarize(number_trips = _.activity_id.nunique())

,number_trips
0,466756


In [18]:
display(HTML("<h4><strong>Number of unique trip taker ids</strong></h4>")) 

In [19]:
r_trips>>summarize(number_trips = _.trip_taker_person_id.nunique())

,number_trips
0,281798


In [20]:
race_ethnicity_by_mode = r_trips>>group_by(_.primary_mode)>>count(_.trip_taker_race_ethnicity)

In [21]:
display(HTML("<h4><strong>Trip Taker Race and Ethnicity by Mode</strong></h4>")) 

In [22]:
chart = (
        alt.Chart(race_ethnicity_by_mode)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("n"),
            color=alt.Color("trip_taker_race_ethnicity", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=race_ethnicity_by_mode["trip_taker_race_ethnicity"].unique().tolist())
            ),
            tooltip=race_ethnicity_by_mode.columns.tolist())
            
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [23]:
display(HTML("<H4><strong>Transit Mode Splits</strong></H4>")) 


In [24]:
display(HTML("<strong>Looking at transit mode, need to alter the transit_agency col and transit_submode to get a better count.</strong>")) 


In [25]:
# ptt_agency_count = (r_trips
#  >>filter(_.primary_mode=="public_transit")
#  >>group_by(_.primary_mode, _.transit_submode, _.transit_agency)
#  >>summarize(n =_.activity_id.nunique())
#  >>arrange(-_.n))

In [26]:
# ptt_agency_count

In [27]:
display(HTML("<h4><strong>Most common transit mode combinations</strong></h4>")) 


In [28]:
# ptt_modes.head(10)

In [29]:
##### unnesting the transit submode and agencies to get counts. 

In [30]:
(r_trips>>filter(_.primary_mode=="public_transit")).sample()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
331444,6855529122147408513,"1 (Tract 2327.01, Los Angeles, CA)","2327.01 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 1286.01, Los Angeles, CA)","1286.01 (Los Angeles, CA)","Los Angeles County, CA",California,public_transit,home,work,16:00:00,18:12:57,132,24.3,unknown_vehicle_type,unknown_fuel_type,"bus, light_rail, bus","Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles","Metro Local Line, Metro E-Line, Metro Rapid Line",civic_institutional,civic_institutional,single_family,single_family,6620640710695100638,14743263179333375648,62.0,male,white_not_hispanic_or_latino,employed,in_person,24516.0,public_transit,3.0,49625.0,two,core,naics62,single_family,not_attending_school,advanced_degree,owner,indo_european,"1 (Tract 1286.01, Los Angeles, CA)","1286.01 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 2327.01, Los Angeles, CA)","2327.01 (Los Angeles, CA)","Los Angeles County, CA",California


In [31]:
agencies_test, mode_test = _utils.get_tranist_agency_counts(r_trips, "primary_mode", "transit_submode", "transit_agency", "activity_id")

In [32]:
agencies_test

,primary_mode,transit_submode,transit_agency,n,agency_count,n_modes_taken
15,public_transit,"bus, bus","Metro - Los Angeles, Metro - Los Angeles",501,1,2
57,public_transit,"bus, bus, bus","Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles",375,1,3
27,public_transit,"bus, bus, bus","Big Blue Bus, Metro - Los Angeles, Metro - Los Angeles",192,2,3
5,public_transit,"bus, bus","Big Blue Bus, Metro - Los Angeles",171,2,2
1,public_transit,bus,Metro - Los Angeles,145,1,1
...,...,...,...,...,...,...
118,public_transit,"rail, bus, bus, bus","Amtrak, Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles",1,2,4
119,public_transit,"rail, bus, bus, rail","Amtrak, Big Blue Bus, Metro - Los Angeles, Amtrak",1,3,4
120,public_transit,"rail, bus, light_rail, bus","Amtrak, Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles",1,2,4
121,public_transit,"subway, bus","Metro - Los Angeles, Metro - Los Angeles",1,1,2


In [33]:
mode_test

,transit_submode,n,n_modes_taken
2,"bus, bus, bus",855,3
1,"bus, bus",842,2
0,bus,163,1
8,"bus, light_rail, bus",124,3
3,"bus, bus, bus, bus",85,4
12,"light_rail, bus",56,2
13,"light_rail, bus, bus",24,3
9,"bus, light_rail, bus, bus",18,4
15,"light_rail, light_rail, bus",12,3
6,"bus, bus, subway",7,3


In [34]:
## This line of code gets all agencies listed, even if agency is named twice. 
## Need code that counts the unique agencies
#ptt_agency_count['agency_count'] = ptt_agency_count.transit_agency.apply(lambda x: len(x.split(", ")))

In [35]:
modes_count = agencies_test>>group_by(_.n_modes_taken)>>summarize(n_trips = _.n.sum())

In [36]:
modes_count

,n_modes_taken,n_trips
0,1,163
1,2,901
2,3,1030
3,4,117


In [37]:
chart = (
        alt.Chart((modes_count))
        .mark_bar(size=60)
        .encode(
            x=alt.X("n_modes_taken", title ="Number of Modes Taken per Trip"),
            y=alt.Y("n_trips", title = "Number of Trips"),
            color=alt.Color("n_trips", title = "Number of Trips",
                            scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=modes_count.columns.tolist())
            
      .properties(title = "How Many Modes are Taken Per Trip",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [38]:
agency_mode_trips = agencies_test>>group_by(_.agency_count, _.n_modes_taken)>>summarize(ntrips=_.n.sum())>>arrange(-_.ntrips)

In [39]:
chart = (
        alt.Chart((agency_mode_trips))
        .mark_circle(size=100)
        .encode(
            x=alt.X("agency_count", title ="Number of Agencies"),
            y=alt.Y("n_modes_taken", title = "Number of Modes Taken"),
            color=alt.Color("ntrips", title = "Number of Trips",
                            scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=agency_mode_trips.columns.tolist())
            
      .properties(title = "How Many Modes are Taken Per Trip",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [40]:
##### Getting columns for each agency and counts 

In [41]:
# ptt_agency_count['transit_agency'].unique().tolist()

In [42]:
agency_list = _utils.get_list_of_agencies(agencies_test, "transit_agency")

In [43]:
# len(agency_list)

In [44]:
display(HTML("<h4><strong>Agencies Identified in Trips Data</strong></h4>")) 

(agency_list)

{'Amtrak',
 'Avta',
 'Beach Cities Transit-City Of Redondo Beach',
 'Big Blue Bus',
 'Culver Citybus',
 'Foothill Transit',
 'Gtrans',
 'La Go Bus',
 'Ladot126',
 'Ladotdt',
 'Ladotmvn',
 'Ladotmvs',
 'Long Beach Transit',
 'Metro - Los Angeles',
 'Metrolink Trains',
 'Santa Clarita Transit'}

In [45]:
### making a copy of the ptt_agency_count
# df = r_trips.copy()

##### Trying ChatGPT approach

In [46]:
# agency_indv_count = (df>>filter(_.primary_mode=="public_transit")>>select(_.transit_agency))
# agency_indv_count['transit_agency'] = agency_indv_count['transit_agency'].astype(str)

In [47]:
# agency_indv_count

In [48]:
df = _utils.get_dummies_by_agency(agencies_test, "transit_agency")

In [49]:
##### Identifying trips with one agency

In [50]:
cols_to_keep = ["transit_submode","unique_agencies", "n","n_modes_taken"]

In [51]:
# chart = (alt.Chart(df)
#              .mark_circle(size=100)
#              .encode(
#                  x=alt.X("n_modes_taken", title="Number of Modes taken"),
#                  y=alt.Y("n", title="Number of Trips"),
#                  color = alt.Color("agency_count", title="Number of Unique Agencies",
#                                   scale=alt.Scale(
#                                       range=cp.CALITP_DIVERGING_COLORS,
#                                       domain=df["agency_count"].unique().tolist())
#             ),
#             tooltip=cols_to_keep)
#         .properties(title = ("Transit Trips Agency Breakdown"), width=500,
#     height=300)
#         )
# chart                    

In [52]:
# (df>>filter(_.agency_count==1)>>arrange(-_.n))

In [53]:
chart = (alt.Chart((df>>filter(_.agency_count==1)))
             .mark_bar()
             .encode(
                 x=alt.X("unique_agencies", title="Agency"),
                 y=alt.Y("n", title="Number of Modes taken"),
                 color = alt.Color("n_modes_taken", title="Number of Trips",
                                  scale=alt.Scale(
                                      range=cp.CALITP_SEQUENTIAL_COLORS,)),
            tooltip=cols_to_keep)
        .properties(title = ("Transit Trips With Only One Agency"), width=800,
    height=500)
        )
chart           

alt.Chart(...)

In [54]:
display(HTML("<h4><strong>Most Common Agency Combination</strong></h4>")) 

(df>>arrange(-_.n)>>select(_.transit_agency)).head(5)

,transit_agency
15,"Metro - Los Angeles, Metro - Los Angeles"
57,"Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles"
27,"Big Blue Bus, Metro - Los Angeles, Metro - Los Angeles"
5,"Big Blue Bus, Metro - Los Angeles"
1,Metro - Los Angeles


In [55]:
df_agencies = _utils.get_agencies_occurances(df)

/home/jovyan/data-analyses/sb125_analyses/corridor_study/_utils.py:98: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_agencies['n_trips'] = df_agencies[list(df_agencies.columns)].sum(axis=1)


In [56]:
df_agencies

,agency,n_trips
0,Culver CityBus,28
1,Foothill Transit,1
2,LA Go Bus,1
3,Metro - Los Angeles,195
4,Amtrak,8
5,AVTA,4
6,GTrans,1
7,Metrolink Trains,1
8,LADOTMVN,54
9,Long Beach Transit,1


In [57]:
tooltip_cols = ["agency", "n_trips"]

In [59]:
chart = (alt.Chart(df_agencies)
        .mark_bar()
        .encode(
            x=alt.X("agency", title = "Agency Name"),
            y=alt.Y("n_trips", title= "Number of boardings reported for trips (One person taking two LA Metro trips will count as 2)"),
            color=alt.Color("n_trips", scale=alt.Scale(range = cp.CALITP_SEQUENTIAL_COLORS)),
            tooltip = tooltip_cols)
      .properties(title = "Number of Times an Agency was used for Trip Taking",
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [60]:
display(HTML("<h4><strong>Trips by Resident Type</strong></h4>")) 


In [61]:
trip_by_res_type = (r_trips
 >>group_by(_.primary_mode,_.trip_taker_resident_type)
 >>summarize(number_trips = _.activity_id.nunique())
 >>arrange(_.primary_mode))

In [62]:
trip_by_res_type

,primary_mode,trip_taker_resident_type,number_trips
0,auto_passenger,core,116685
1,auto_passenger,donut,2
2,auto_passenger,visitor,28818
3,commercial,NaN,10154
4,on_demand_auto,core,3191
5,on_demand_auto,visitor,595
6,private_auto,core,294704
7,private_auto,donut,1
8,private_auto,visitor,10394
9,private_auto,NaN,1


In [63]:
chart = (
        alt.Chart(trip_by_res_type)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode", title = "Mode"),
            y=alt.Y("number_trips", title = "Number of Trips"),
            color=alt.Color("trip_taker_resident_type", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=trip_by_res_type["trip_taker_resident_type"].unique().tolist())
            ),
            tooltip=trip_by_res_type.columns.tolist())
      .properties(title = "Trips by Resident Type",
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [64]:
display(HTML("<h4><strong>Unique Household Ids</strong></h4>")) 

In [65]:
r_trips>>count(_.trip_taker_household_id)>>arrange(-_.n)

,trip_taker_household_id,n
160790,,33634
267303,NaN,16384
79436,9146817053558612433,16
23626,2709972826117271852,15
30303,3488167635549658463,14
...,...,...
267296,9999130854496671765,1
267299,999930961444594419,1
267300,9999534358623422158,1
267301,9999886348099258237,1


In [66]:
display(HTML("<h4><strong>Checking one household id</strong></h4>")) 


In [67]:
print(len(r_trips>>filter(_.trip_taker_household_id == 9146817053558612433)))
(r_trips>>filter(_.trip_taker_household_id == 9146817053558612433)).sample()

16


,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
40159,7958386252736501273,"1 (Tract 2623.01, Los Angeles, CA)","2623.01 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 69, Ventura, CA)","69 (Ventura, CA)","Ventura County, CA",California,private_auto,eat,home,11:18:00,12:03:59,45,29.6,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,single_family,single_family,retail,retail,8423484308586815884,9146817053558612433,32.0,female,asian_not_hispanic_or_latino,employed,in_person,17261.0,private_auto,11.0,318660.0,three_plus,core,naics812199,single_family,not_attending_school,some_college,owner,asian_pacific,"1 (Tract 2623.01, Los Angeles, CA)","2623.01 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 7010, Los Angeles, CA)","7010 (Los Angeles, CA)","Los Angeles County, CA",California


In [68]:
## checking one household id
r_trips>>filter(_.trip_taker_household_id == 9146817053558612433)>>count(_.trip_taker_person_id, _.trip_taker_age,
                                                                         _.trip_taker_sex, _.trip_taker_household_size)

,trip_taker_person_id,trip_taker_age,trip_taker_sex,trip_taker_household_size,n
0,4508016952300979367,25.0,female,11.0,5
1,4736398356956282212,38.0,female,11.0,2
2,8423484308586815884,32.0,female,11.0,2
3,12395012647665989305,21.0,female,11.0,1
4,14673592671547908751,18.0,female,11.0,4
5,16545634957747565398,49.0,female,11.0,1
6,16684115452596066816,53.0,male,11.0,1


In [69]:
display(HTML("<h4><strong>Number of Travelers by Resident Type: Replica Trips Data</strong></h4>")) 

In [70]:
r_trips>>group_by(_.trip_taker_resident_type)>>summarize(_unique_ids = _.trip_taker_person_id.nunique())

,trip_taker_resident_type,_unique_ids
0,core,246597
1,donut,2
2,visitor,35198
3,NaN,1


In [71]:
display(HTML("<h4><strong>Traveler Demographics: Replica Trips Data</strong></h4>")) 
display(HTML("Fitered for Core Residents")) 
(r_trips
 >>filter(_.trip_taker_home_bgrp_2020!="Visitor (no home location)")
 >>group_by(_.primary_mode, _.trip_taker_sex)
 >>summarize(
     n_ppl = _.trip_taker_person_id.nunique(),
     avg_h_income = _.trip_taker_household_income.mean(),
     avg_p_income = _.trip_taker_individual_income.mean(),
     avg_age = _.trip_taker_age.mean())
)
     

,primary_mode,trip_taker_sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,44266,162899.902386,57807.907035,43.646941
1,auto_passenger,male,43836,173439.458096,87237.704327,42.863541
2,commercial,NaN,1,NaN,NaN,NaN
3,on_demand_auto,female,1560,181325.515908,58599.185901,46.576419
4,on_demand_auto,male,1532,181278.142947,92035.743703,43.226700
5,private_auto,female,90634,167383.189351,62989.923621,43.786498
6,private_auto,male,99967,174477.157845,94470.932013,43.811427
7,private_auto,NaN,1,NaN,NaN,NaN
8,public_transit,female,927,81553.446911,39058.572394,42.898649
9,public_transit,male,979,97733.906166,44295.896336,39.824844
